In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("test").master("local[*]").getOrCreate()

In [6]:
training_df = spark.read.format("csv").option("inferschema", True).option("header", True)\
.load("DataDummy/data/DataBricks_Training.csv")

In [10]:
training_df.show(2)

+---+---------+----+-------+-----------+--------+
| ID|     Name| Fee|  Venue|       Date|Duration|
+---+---------+----+-------+-----------+--------+
|  1|HE Hadoop|9000| Mumbai|01-Aug-2018|       2|
|  2| HE Spark|7000|Kolkata|04-Aug-2018|       3|
+---+---------+----+-------+-----------+--------+
only showing top 2 rows



In [14]:
training_double_df = spark.read.format("csv").option("sep", "|").option("inferschema", True).option("header", True)\
.load("DataDummy/dataframe_sample.csv")

In [15]:
training_double_df.show(2)

+---+-------------------+-------------------+--------+
| id|           end_date|         start_date|location|
+---+-------------------+-------------------+--------+
|  1|2015-10-14 00:00:00|2015-09-14 00:00:00|   CA-SF|
|  2|2015-10-15 01:00:20|2015-08-14 00:00:00|   CA-SD|
+---+-------------------+-------------------+--------+
only showing top 2 rows



In [16]:
training_df.join(training_double_df.hint("id"), "ID").show()

+---+-------------------+-----+---------+-----------+--------+-------------------+-------------------+--------+
| ID|               Name|  Fee|    Venue|       Date|Duration|           end_date|         start_date|location|
+---+-------------------+-----+---------+-----------+--------+-------------------+-------------------+--------+
|  1|          HE Hadoop| 9000|   Mumbai|01-Aug-2018|       2|2015-10-14 00:00:00|2015-09-14 00:00:00|   CA-SF|
|  2|           HE Spark| 7000|  Kolkata|04-Aug-2018|       3|2015-10-15 01:00:20|2015-08-14 00:00:00|   CA-SD|
|  3|        HE SparkSQL| 6000|Hyderabad|07-Aug-2018|       4|2015-10-16 02:30:00|2015-01-14 00:00:00|   NY-NY|
|  4|     HE Spark Graph| 6000|  Chennai|10-Aug-2018|       5|2015-10-17 03:00:20|2015-02-14 00:00:00|   NY-NY|
|  5|HE Machine Learning|10000|   London|13-Aug-2018|       2|2015-10-18 04:30:00|2014-04-14 00:00:00|   CA-SD|
+---+-------------------+-----+---------+-----------+--------+-------------------+-------------------+--

In [20]:
training_df.join(training_double_df.hint("id"), "ID").explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(ID))
:- Relation[ID#57,Name#58,Fee#59,Venue#60,Date#61,Duration#62] csv
+- Relation[id#193,end_date#194,start_date#195,location#196] csv

== Analyzed Logical Plan ==
ID: int, Name: string, Fee: int, Venue: string, Date: string, Duration: int, end_date: timestamp, start_date: timestamp, location: string
Project [ID#57, Name#58, Fee#59, Venue#60, Date#61, Duration#62, end_date#194, start_date#195, location#196]
+- Join Inner, (ID#57 = id#193)
   :- Relation[ID#57,Name#58,Fee#59,Venue#60,Date#61,Duration#62] csv
   +- Relation[id#193,end_date#194,start_date#195,location#196] csv

== Optimized Logical Plan ==
Project [ID#57, Name#58, Fee#59, Venue#60, Date#61, Duration#62, end_date#194, start_date#195, location#196]
+- Join Inner, (ID#57 = id#193)
   :- Filter isnotnull(ID#57)
   :  +- Relation[ID#57,Name#58,Fee#59,Venue#60,Date#61,Duration#62] csv
   +- Filter isnotnull(id#193)
      +- Relation[id#193,end_date#194,start_date#195,loca

In [21]:
print(spark.conf.get("spark.sql.autoBroadcastJoinThreshold"))

10485760
